In [6]:
from sklearn.datasets import fetch_openml
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

# dataset = fetch_openml(name="California-Housing-Classification", parser="auto")
# low_class_name = 'False'
# high_class_name = 'True'

dataset = fetch_openml(name="spambase", parser="auto")
low_class_name = '0'
high_class_name = '1'

# Really, we should shuffle the data to be on the safe side
X = dataset.data.values.astype(float)
Y = dataset.target.to_numpy()
Y = (Y == high_class_name).astype(int) - (Y == low_class_name).astype(int)

X, Y = unison_shuffled_copies(X, Y)

N = 1000
X = X[:N]
Y = Y[:N]

In [7]:
from classifiers import ConformalPassiveAggressive, OnlineConfidencePrediction, ConformalOneNearestNeighbours, ConformalPassiveAggressive_I

# cp = ConformalPassiveAggressive(X.shape[1])
# cp = ConformalPassiveAggressive_I(X.shape[1], 0.001)
cp = ConformalOneNearestNeighbours()


Epsilon = np.linspace(0.05, 0.5, num=10, endpoint=True)
protocol = OnlineConfidencePrediction(cp, Epsilon)

for i, (x, y) in tqdm(enumerate(zip(X, Y)), total=X.shape[0]):
    # Reality revieals x
    # Predictor predicts set and p-values
    p_values = protocol.predict_p(x)
    Gammas = protocol.compute_Gamma(p_values)

    # Reality revieals y
    protocol.learn_one(x, y)

    # Update metrics
    protocol.update_metrics(p_values, Gammas, y)




  0%|          | 0/1000 [00:00<?, ?it/s]

/home/johan/Documents/PhD/PREMACOP/OnlineConformalPrediction/classifiers.py:373: RuntimeWarning: divide by zero encountered in divide
  Alpha = np.nan_to_num(same_label_distances / different_label_distances, nan=np.inf)
/home/johan/Documents/PhD/PREMACOP/OnlineConformalPrediction/classifiers.py:373: RuntimeWarning: invalid value encountered in divide
  Alpha = np.nan_to_num(same_label_distances / different_label_distances, nan=np.inf)


In [8]:
print('Observed average Error')
for epsilon, Err in protocol.Err.items():
    print(f'{epsilon}: {Err / Y.shape[0]}')

Observed average Error
0.05: 0.053
0.1: 0.103
0.15000000000000002: 0.153
0.2: 0.202
0.25: 0.26
0.3: 0.314
0.35000000000000003: 0.352
0.4: 0.397
0.45: 0.44
0.5: 0.498


In [9]:
print('Average Observed Excess')
for epsilon, Err in protocol.OE.items():
    print(f'{epsilon}: {Err / Y.shape[0]}')

Average Observed Excess
0.05: 0.741
0.1: 0.614
0.15000000000000002: 0.516
0.2: 0.422
0.25: 0.341
0.3: 0.288
0.35000000000000003: 0.243
0.4: 0.205
0.45: 0.182
0.5: 0.158


In [10]:
protocol.OF

234.40546753304596